# Housing
**Tivoli Model One** has a nice housing, unfortunately even broken models still cost a lot of money (€65/pce).  There are cheaper, lower quality clones available :
* Nedis König HAV-TR13
* ...

# Display
## Options
### LCD/TFT
* Cheaper
* RGB
* integrated touch

### OLED
* single color, choose white to match the silk screen
* Doesn't need a backlight, so should have more contrast than LCD.  When it's dark, the dark pixels are off, no backlight to shine through.
* [2.42" OLED Display 128x64 pixels - White, SKU: 004993](https://www.tinytronics.nl/shop/en/displays/oled/2.42-inch-oled-display-128*64-pixels-white-i2c)
* [Adafruit Monochrome 2.42" 128x64 OLED Graphic Display Module Kit Product ID: 2719 ](https://www.adafruit.com/product/2719)
* [Waveshare 25742](https://www.waveshare.com/2.42inch-oled-module.htm?sku=25742)
* [Focus O12864B-GW-TW3](https://www.digikey.be/en/products/detail/focus-lcds/O12864B-GW-TW3/12691219)

# Power source
## Options
### DC Barrel Jack
* cheap (connectors, cables, adapters)
* not much hardware needed
* no control over input voltage 

### USB-C
#### References
* [A Primer on USB Type-C® and USB Power Delivery Applications and Requirements](https://www.ti.com/lit/wp/slyy109b/slyy109b.pdf)

#### Options
##### USB-PD 1.0 or higher (if more than 5V is needed)
* USB-PD (Power Delivery) IC needed: https://hackaday.com/2020/10/23/a-plethora-of-power-delivery-potential/
* Onsemi
    * [FUSB302](https://hackaday.io/project/176680-pd-micro-usb-c-pd30-pps-trigger)
    * [USB-PD-Breakout](https://github.com/ReclaimerLabs/USB-PD-Breakout) : source code
    * [Open-Source Firmware for ZY12PDN USB-PD](https://github.com/manuelbl/zy12pdn-oss)
    * [Notes on USB PD Triggers (And ZY12PDN Instructions)](https://www.alexwhittemore.com/notes-on-usb-pd-triggers-and-zy12pdn-instructions/)
    * [Fusb302mpx](https://jlcpcb.com/partdetail/Onsemi-Fusb302mpx/C442699)
    * [Pinecil schematic](https://files.pine64.org/doc/Pinecil/Pinecil_schematic_v2.0_20220608.pdf)
* Microchip: no solution for USB-C
* No QFN24 pin compatible solution
* Diodes : 
    * [AP33771] : QFN24
    * [AP33772] : QFN24
* Infineon
    * CYPD3177-24LQXQT : QFN24
* Rohm, TI : no solution wit 24 pins or less
* ST
    * [STUSB4500](https://hackaday.com/2021/04/21/easy-usb%E2%80%91c-power-for-all-your-devices/) : QFN24
    * [STUSB4500QTR](https://jlcpcb.com/partdetail/Stmicroelectronics-STUSB4500QTR/C2678061) : $2.1,  220pcs
* NXP
    * [PTN5110] : QFN16 2.6x2.6mm
* Richtek
    * [RT1711P]
* Injoinic
    * [IP2721](https://hackaday.io/project/173717-usb-c-pd-max-trigger-board) : TSSOP16
    
#####  UFP USB 2.0 without USB PD (5V only & ≤15 W)
* [TUSB320LAIRWBR]()
* [WUSB3801Q-12/TR]
* [NXP PTN5150A]
* [Onsemi FUSB303B]
* [Diodes PI5USB30216CXUAEX]

#### Choice
**FUSB302** : cheap, small, available, open source firmware

# Development kit
## Options
### AI-Thinker ESP32 Audio Kit
* ESP32-A1S module has internal I2C, unfortunately it's connected to external pins on the development board.
* Very limited pinout
* PSRAM doesn't work

### ESP32-LyraT-Mini
* [website](https://docs.espressif.com/projects/esp-adf/en/latest/design-guide/dev-boards/get-started-esp32-lyrat-mini.html)
* Uses ES8311 audio codec ($0.44)

## Audio codec
There's an ES8388 audio codec on the board.  It's connected to the ESP32 via I2S.  The ES8388 is controlled via I2C.  There's noticeable noise on both the headphone output and the power amplifier.  There's also sometimes creaking sound, which disappears often when restarting the stream.

# Firmware
## Toolchain
### Options
#### ESP-IDF

#### PlatformIO with Arduino

### Choice
PlatformIO with Arduino : I've made a working web radio with this toolchain, so I'm familiar with it.

## Web radio
* [ESP32-audiokit-webradio-webinterface](https://github.com/thieu-b55/ESP32-audiokit-webradio-webinterface/tree/main) : badly written code, but leads me on the path to an ES8388 driver.
* [ESP32-MiniWebRadio](https://github.com/schreibfaul1/ESP32-MiniWebRadio)
* [ESP32_WebRadio ](https://github.com/michelep/ESP32_WebRadio) : streams saved as a json file. All code in a single .ino file.

### Libraries

#### Remote monitoring
[Husarnet](https://husarnet.com/) : remote access to ESP32.  It's free for non-commercial use (up to 5 devices).  This is optional, but it's nice to be able to monitor the device remotely.

Several APIs are available :
* [SimpleCLI](https://github.com/SpacehuhnTech/SimpleCLI) with Husarnet-server: send custom commands over TCP-socket to ESP32.  We might send JSON or XML over the socket to control the device.
* Websockets : bi-directional, full-duplex communication over a persistent TCP connection, stateful
  * [ESP8266WebCLI](https://github.com/SpacehuhnTech/SimpleCLI/blob/master/examples/ESP8266WebCLI/ESP8266WebCLI.ino) : simple web interface to send commands to ESP32
* [Telnet](https://github.com/LennartHennigs/ESPTelnet) : You can send and receive strings over the telnet connection.  We might send JSON or XML over the socket to control the device.
* RESTful HTTP : HTTP is a request-response based protocol, stateless, using non-persistent TCP connections
```json
curl --header "Content-Type: application/json" \
  --request POST \
  --data '{"username":"xyz","password":"xyz"}' \
  http://localhost:3000/api/login
```

#### Stream decoding
* [ESP32-audioI2S](https://github.com/schreibfaul1/ESP32-audioI2S) : I2S audio library for the ESP32
  * no need to pass in media type
  * shows song title
  * allows to quickly switch between streams
* ~~[arduino-audiokit](https://github.com/pschatzmann/arduino-audiokit)~~ : crashes or produces junk audio when switching between MP3 and AAC streams

#### O-LED
* [Adafruit_SSD1305](https://github.com/adafruit/Adafruit_SSD1305) : works
* [SSD1306Ascii](https://github.com/greiman/SSD1306Ascii/) : used by ArduinoMenu-library, only text support (probably subset from Adafruit library), works

#### Audio codec ES8388
Mix of following libraries :
* [es8388arduino](https://github.com/vanbwodonk/es8388arduino)
* [es8388](https://github.com/maditnerd/es8388)
* [arduino-audiokit](https://github.com/pschatzmann/arduino-audiokit)

#### Knob / Rotary encoder
* paulstoffregen/Encoder @ ^1.4.2 : works with Husarnet version of espressif32 platform.  Crashes with official version of espressif32 platform.
  alternative rotary encoder library : mathertel/RotaryEncoder (used on https://github.com/LieBtrau/better-alarm-clock/blob/master/firmware/full-application/lib/TactileElements/RotaryEncoder_Tactiles.cpp)

#### Buttons
  thomasfredericks/Bounce2 @ ^2.71

#### Menu library
There are many libraries for LCD displays, but not many for OLED displays.  The following libraries support OLED displays :
* [ArduinoMenu](https://github.com/neu-rah/ArduinoMenu) : supports rotary encoder and OLED
* [TcMenu](https://www.thecoderscorner.com/products/arduino-libraries/tc-menu/tcmenu-plugins/adafruit_gfx-renderer-plugin/) : looks complex

#### Other
  stevemarple/AsyncDelay @ ^1.1.2


# Radio stations
When no file format is given, execute a ```wget``` with the url to get the content type.

## Belgium
* [JoeFM](http://icecast-qmusicbe-cdp.triple-it.nl/joe.mp3)
* [QMusic](http://icecast-qmusicbe-cdp.triple-it.nl/qmusic.mp3)
* [Radio Nostalgie](http://25633.live.streamtheworld.com/NOSTALGIEWHATAFEELING.mp3)
* [MNM](http://vrt.streamabc.net/vrt-mnm-mp3-64-4492886) : audio/mpeg
* [VRT NWS](http://progressive-audio.vrtcdn.be/content/fixed/11_11niws-snip_lo.mp3)

## Philippines
* [Bespren Radio](http://streamnavs.net:8089/live) : audio/aacp
* [Tunog Pinoy Power FM](http://streamnavs.net:8143/live) : audio/aacp
* [Win Radio 107.9 Naga](http://streamnavs.net:8161/stream) : audio/aacp
* [94.7 Happy Radio](http://stream-152.zeno.fm/2xk9xz3wpl2vv?zs=Pb9DJKxNRxmKiS4Pa6O9SQ) : audio/mpeg
* [DWAY Care 104.3](http://sg-icecast.eradioportal.com:8000/febc_dway) : audio/mpeg
* [Love Radio](http://loveradiolegaspi.radioca.st/;) : audio/mpeg
* [Barangay LS97.1](https://magic.radioca.st/stream) : audio/mpeg
* [Hugot Radio 2.0](http://cast3.asurahosting.com/proxy/istreamo/stream) : audio/mpeg
* [OK FM Legazpi](http://server10.reliastream.com:8063/xstream)